# Road Damage YOLOv8 Train and Predict

In [3]:
#!rm -rf 

In [4]:
!pip install wandb
!wandb --version

wandb, version 0.12.21


In [5]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.7/616.7 kB 16.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 16.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 614.9/614.9 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 614.4/614.4 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.9/606.9 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.2/606.2 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.5/605.5 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.6/605.6 kB 33.2 MB/s eta 0:00:00
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.5.4.60
    Uninstalling opencv-python-4.5.4.60:
      Successfully uninstalled opencv-python-4.5.4.60


In [6]:
import os
import cv2
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import shutil
from shutil import copyfile
import matplotlib.pyplot as plt
from matplotlib import animation, rc
rc('animation', html='jshtml')
from PIL import Image
import ultralytics
from ultralytics import YOLO
ultralytics.checks()

Ultralytics YOLOv8.0.145 🚀 Python-3.7.12 torch-1.11.0+cpu CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5934.1/8062.4 GB disk)


In [7]:
import wandb

# Replace 'your_api_key' with your actual W&B API key
wandb.login(key="e8eb8ff45f7c61e76a3e59c979954d89816ac35a")


wandb: Currently logged in as: asminparikh (asminparikh-svkm-s-narsee-monjee-institute-of-management). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [9]:
import os

# Define paths
train_path = 'datasets/train/'
valid_path = 'datasets/valid/'
test_path = 'datasets/test/'

# Create directories if they don't exist
os.makedirs(train_path, exist_ok=True)
os.makedirs(valid_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)

print("Directories are set up:")
print(f"Train path: {train_path}")
print(f"Valid path: {valid_path}")
print(f"Test path: {test_path}")



Directories are set up:
Train path: datasets/train/
Valid path: datasets/valid/
Test path: datasets/test/


In [6]:
import os
import shutil

# Define paths
base_path = '/kaggle/input/road-damage-tracking-dataset-rdtd-v10/train-pure/train-pure'
train_path = 'datasets/train/'
valid_path = 'datasets/valid/'
test_path = 'datasets/test/'

# Create directories if they don't exist
os.makedirs(train_path, exist_ok=True)
os.makedirs(valid_path, exist_ok=True)
os.makedirs(test_path, exist_ok=True)

# Helper function to copy files
def copy_files(indices, destination_path):
    for i in indices:
        ano_path = ano_paths[i]
        # Copy annotation file
        shutil.copy(ano_path, destination_path)
        
        # Generate corresponding image path and check if it exists
        img_filename = os.path.basename(ano_path).replace('.txt', '.jpg')
        img_path = os.path.join(base_path, 'images', img_filename)
        
        if os.path.exists(img_path):
            shutil.copy(img_path, destination_path)
        else:
            print(f"Image file not found: {img_path}")

# Copy files to train, valid, and test directories
copy_files(train_i, train_path)
copy_files(valid_i, valid_path)
copy_files(test_i, test_path)

# Print counts of copied files
print(f"Train files: {len(os.listdir(train_path))}")
print(f"Valid files: {len(os.listdir(valid_path))}")
print(f"Test files: {len(os.listdir(test_path))}")


Train files: 480
Valid files: 160
Test files: 160


# Data Preparation

In [4]:
import os
import random
ano_paths=[]
i=0
for dirname, _, filenames in os.walk('/kaggle/input/road-damage-tracking-dataset-rdtd-v10/train-pure/train-pure/labels'):
    for filename in filenames:
        if filename[-4:]=='.txt' and i<400:
            ano_paths+=[(os.path.join(dirname, filename))]
            i+=1
        
n=len(ano_paths) 
print(n)
N=list(range(n))
random.shuffle(N)

train_ratio = 0.6
valid_ratio = 0.2
test_ratio = 0.2

train_size = int(train_ratio*n)
valid_size = int(valid_ratio*n)

train_i = N[:train_size]
valid_i = N[train_size:train_size+valid_size]
test_i = N[train_size+valid_size:]

#print(train_i)
#print(valid_i)
#print(test_i)

400


In [7]:
for i in train_i:
    ano_path = ano_paths[i]
    img_path = os.path.join('/kaggle/input/road-damage-tracking-dataset-rdtd-v10/train-pure/train-pure/images',
                            ano_path.split('/')[-1][0:-4] + '.jpg')
    print(f"Attempting to copy: {ano_path} and {img_path}")  # Debug print to check paths
    
    try:
        # Copy annotation file
        !cp {ano_path} {train_path}
        
        # Copy corresponding image file
        !cp {img_path} {train_path}
        
    except Exception as e:
        print(f"Error copying {ano_path} or {img_path}: {e}")
        continue

# Print the number of files in the training path
print(f"Total files in train_path: {len(os.listdir(train_path))}")


Attempting to copy: /kaggle/input/road-damage-tracking-dataset-rdtd-v10/train-pure/train-pure/labels/India_006581_jpg.rf.754bd3a5ba5f002ad29ffeccb62ac332.txt and /kaggle/input/road-damage-tracking-dataset-rdtd-v10/train-pure/train-pure/images/India_006581_jpg.rf.754bd3a5ba5f002ad29ffeccb62ac332.jpg
Attempting to copy: /kaggle/input/road-damage-tracking-dataset-rdtd-v10/train-pure/train-pure/labels/Japan_000413_jpg.rf.d1eb6b78ee9a6df8c19a495d0d5bdb8a.txt and /kaggle/input/road-damage-tracking-dataset-rdtd-v10/train-pure/train-pure/images/Japan_000413_jpg.rf.d1eb6b78ee9a6df8c19a495d0d5bdb8a.jpg
Attempting to copy: /kaggle/input/road-damage-tracking-dataset-rdtd-v10/train-pure/train-pure/labels/India_002066_jpg.rf.06cf9e38e26a0202458194ab9934d90a.txt and /kaggle/input/road-damage-tracking-dataset-rdtd-v10/train-pure/train-pure/images/India_002066_jpg.rf.06cf9e38e26a0202458194ab9934d90a.jpg
Attempting to copy: /kaggle/input/road-damage-tracking-dataset-rdtd-v10/train-pure/train-pure/labels

In [8]:
for i in valid_i:
    ano_path=ano_paths[i]
    img_path=os.path.join('/kaggle/input/road-damage-tracking-dataset-rdtd-v10/train-pure/train-pure/images',
                          ano_path.split('/')[-1][0:-4]+'.jpg')
    try:
        !cp {ano_path} {valid_path}
        !cp {img_path} {valid_path}
    except:
        continue
print(len(os.listdir(valid_path)))

160


In [9]:
for i in test_i:
    ano_path=ano_paths[i]
    img_path=os.path.join('/kaggle/input/road-damage-tracking-dataset-rdtd-v10/train-pure/train-pure/images',
                          ano_path.split('/')[-1][0:-4]+'.jpg')
    try:
        !cp {ano_path} {test_path}
        !cp {img_path} {test_path}
    except:
        continue
print(len(os.listdir(test_path)))      

160


# Create yaml file 

In [10]:
import yaml

data_yaml = dict(
    train ='train',
    val ='valid',
    test='test',
    nc =1,
    names =['damage']
)

with open('data.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)
    
%cat data.yaml

{names: [damage], nc: 1, test: test, train: train, val: valid}


In [16]:
names =['damage']
M=list(range(len(names)))
class_map=dict(zip(M,names))

# Train

!pip show ultralytics


In [19]:
!pip show ultralytics


In [22]:
!pip install virtualenv
!virtualenv yolov8_env
!source yolov8_env/bin/activate


created virtual environment CPython3.7.12.final.0-64 in 751ms
  creator CPython3Posix(dest=/kaggle/working/yolov8_env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==22.3.1, setuptools==65.6.3, wheel==0.38.4
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [23]:
!pip install ultralytics --no-deps
!pip install tensorflow==2.9.0 numpy==1.19.5 protobuf==3.20.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.7/511.7 MB 1.4 MB/s eta 0:00:0000:0100:05
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 60.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.9 MB/s eta 0:00:00
ERROR: Cannot install numpy==1.19.5 and tensorflow==2.9.0 because these package versions have conflicting dependencies.

The conflict is caused by:
    The user requested numpy==1.19.5
    tensorflow 2.9.0 depends on numpy>=1.20

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip attempt to solve the dependency conflict

ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [ ]:
import torch
from pathlib import Path
from torchvision import transforms
from PIL import Image

# Load the YOLOv8 model (replace with the appropriate path to your model weights)
model_path = "path/to/yolov8x.pt"
model = torch.load(model_path, map_location=torch.device('cpu'))  # or use 'cuda' if you have a GPU
model.eval()

# Load an image for inference
img_path = "/kaggle/input/road-damage-tracking-dataset-rdtd-v10/train-pure/train-pure/images/abc123.jpg"
img = Image.open(img_path).convert("RGB")

# Preprocess the image (resize and normalize, this depends on how the model was trained)
preprocess = transforms.Compose([
    transforms.Resize((640, 640)),  # Resize to the expected input size of the model
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Example normalization values
])
input_tensor = preprocess(img).unsqueeze(0)  # Add batch dimension

# Perform inference
with torch.no_grad():
    outputs = model(input_tensor)  # Perform inference

# Process the outputs (example: get detections)
# The output processing step will depend on the model's implementation and output format
# You would typically get bounding boxes, confidences, and class predictions

# Example: Assuming output is in a tensor format
boxes, confidences, class_ids = outputs  # Placeholder - adjust depending on model

# Do post-processing to filter detections (e.g., NMS)
# Then visualize or save the detections as per your requirements


In [ ]:
!yolo task=detect mode=train model=yolov8x.pt data=data.yaml epochs=12 imgsz=480

# Result of Training

In [ ]:
paths2=[]
for dirname, _, filenames in os.walk('/kaggle/working/runs/detect/train'):
    for filename in filenames:
        if filename[-4:]=='.jpg':
            paths2+=[(os.path.join(dirname, filename))]
paths2=sorted(paths2)

In [ ]:
for path in paths2:
    image = Image.open(path)
    image=np.array(image)
    plt.figure(figsize=(20,10))
    plt.imshow(image)
    plt.show()

# Predict

In [ ]:
best_path0='runs/detect/train/weights/best.pt'
source0='datasets/test'

In [ ]:
ppaths=[]
for dirname, _, filenames in os.walk(source0):
    for filename in filenames:
        if filename[-4:]=='.jpg':
            ppaths+=[(os.path.join(dirname, filename))]
ppaths=sorted(ppaths)
print(ppaths[0])
print(len(ppaths))

In [ ]:
model2 = YOLO(best_path0)

In [ ]:
!yolo task=detect mode=predict model={best_path0} conf=0.1 source={source0}

# Result of Prediction

In [ ]:
results = model2.predict(source0,conf=0.1)
print(len(results))

In [ ]:
print((results[0].boxes.data))

In [ ]:
PBOX=pd.DataFrame(columns=range(6))
for i in range(len(results)):
    arri=pd.DataFrame(results[i].boxes.data.cpu().numpy()).astype(float)
    path=ppaths[i]
    file=path.split('/')[-1]
    arri=arri.assign(file=file)
    arri=arri.assign(i=i)
    PBOX=pd.concat([PBOX,arri],axis=0)
PBOX.columns=['x','y','x2','y2','confidence','class','file','i']
display(PBOX)

In [ ]:
PBOX['class']=PBOX['class'].apply(lambda x: class_map[int(x)])
PBOX=PBOX.reset_index(drop=True)
display(PBOX)
display(PBOX['class'].value_counts())

In [ ]:
def draw_box2(n0):
    
    ipath=ppaths[n0]
    image=cv2.imread(ipath)
    H,W=image.shape[0],image.shape[1]
    file=ipath.split('/')[-1]
    
    if PBOX[PBOX['file']==file] is not None:
        box=PBOX[PBOX['file']==file]
        box=box.reset_index(drop=True)
        #display(box)

        for i in range(len(box)):
            label=box.loc[i,'class']
            x=int(box.loc[i,'x'])
            y=int(box.loc[i,'y'])
            x2=int(box.loc[i,'x2']) 
            y2=int(box.loc[i,'y2'])
            #print(label,x,y,x2,y2)
            #cv2.putText(image, f'{label}', (x, int(y-4)), 
            #            cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,255,0),2)
            cv2.rectangle(image,(x,y),(x2,y2),(0,255,0),2) #green
    
    #plt.imshow(image)
    #plt.show()   
    
    return image

In [ ]:
def create_animation(ims):
    fig=plt.figure(figsize=(12,8))
    im=plt.imshow(cv2.cvtColor(ims[0],cv2.COLOR_BGR2RGB))
    text = plt.text(0.05, 0.05, f'Slide {0}', transform=fig.transFigure, fontsize=14, color='blue')
    plt.axis('off')
    plt.close()

    def animate_func(i):
        im.set_array(cv2.cvtColor(ims[i],cv2.COLOR_BGR2RGB))
        text.set_text(f'Slide {i}')        
        return [im]    
    
    return animation.FuncAnimation(fig, animate_func, frames=len(ims), interval=1000)

In [ ]:
images2=[]
for i in tqdm(range(len(ppaths))):
    images2+=[draw_box2(i)]

In [ ]:
create_animation(images2)